In [8]:
# Imports
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [9]:
# Create Engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [10]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

# Declare a Base object here
Base = declarative_base()

In [11]:
# Define the ORM class for `Measurements`
class Measurement(Base):
    
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [12]:
# Define the ORM class for `Station`
class Station(Base):
    
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [13]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)

In [14]:
# Verify that the table names exist in the database
engine.table_names()

['measurement', 'station']

In [15]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Station.__table__, 'clean_hawaii_stations.csv')
populate_table(engine, Measurement.__table__, 'clean_hawaii_measurements.csv')

In [16]:
# Use a basic query to validate that the data was inserted correctly for table `measurement`
engine.execute("SELECT * FROM measurement LIMIT 1").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0)]

In [17]:
# Use a basic query to validate that the data was inserted correctly for table `station`
engine.execute("SELECT * FROM station LIMIT 1").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)]